In [42]:
# DATA READ - REYNOLDS AND VELOCITY VALUES
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#import data

ANNDF = pd.read_excel('fxReynoldsV3.xlsx')

X = ANNDF.iloc[:,[1,3]].values
Y = ANNDF.iloc[:, 2].values.reshape(-1,1)

xtrain, xtest, ytrain,  ytest = train_test_split(X, Y, train_size = 0.75)

scalerx = StandardScaler()
scalery = StandardScaler()

Xtrain = scalerx.fit_transform(xtrain)
Ytrain = scalery.fit_transform(ytrain)



mlp = MLPRegressor(max_iter=5000)



In [43]:

# Define the KFold cross-validation
kf = KFold(n_splits=10, shuffle=True)

# Define the parameter grid
parameters = {
    'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (8, 8), (4, 4, 4), (6, 6, 6), (8, 8, 8), (4, 4, 4, 4), (6, 6, 6, 6), (8, 8, 8, 8)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam']
}

# Define GridSearchCV
gsearchcv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=kf)

# Fit the model
gsearchcv.fit(Xtrain, Ytrain)

# Get best score and parameters
bestac = gsearchcv.best_score_ * 100  # Convert to percentage
bestparam = gsearchcv.best_params_

print('Best Accuracy: {:.2f} %'.format(bestac))
print('Best Parameters:', bestparam)


c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

Best Accuracy: 99.07 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (8, 8, 8, 8), 'solver': 'adam'}


In [44]:
z = (scalerx.transform(xtest))
print(scalerx.inverse_transform(z))

[[2.60338219e+03 4.16200000e-01]
 [4.97788877e+03 4.33100000e-01]
 [1.53919836e+03 3.32600000e-01]
 ...
 [4.65568104e+03 4.33100000e-01]
 [2.98877124e+03 3.32600000e-01]
 [3.54380712e+03 3.74500000e-01]]


In [45]:
input_data = np.array([[1843, 0.4331]])

# Transformar os dados de entrada usando o scaler treinado
input_data_scaled = scalerx.transform(input_data)

# Fazer a previsão com o modelo treinado
predicted_scaled = gsearchcv.predict(input_data_scaled)

# Inverter a transformação do output (scaling)
predicted = scalery.inverse_transform(predicted_scaled.reshape(-1, 1))

Xtest_scaled = scalerx.transform(xtest)

ypred = gsearchcv.predict(Xtest_scaled).reshape(-1,1)

ypred_or = scalery.inverse_transform(ypred)

# Mostrar o resultado
print('Predicted value for [2000, 0.24]:', predicted[0][0])


Predicted value for [2000, 0.24]: 0.5137381414643231


In [123]:
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo

# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(1000, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# Prever os valores de 'f' usando a ANN nos pontos da grade
f_pred_scaled = gsearchcv.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

# Calcular a superfície usando a fórmula fornecida
f_formula = (4.3059 / np.log(Re_grid + 3.1918)) + ((0.30649 / Stdp_grid) - 0.80937)

# Criar o gráfico 3D para os resultados da ANN com contornos
surface_ann = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_pred_grid, 
    colorscale='Viridis',
    name='Superfície ANN',
    colorbar=dict(title='ANN f', len=0.5, x=0.9),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_pred_grid),
            "end": np.max(f_pred_grid),
            "size": (np.max(f_pred_grid) - np.min(f_pred_grid)) / 20,  # Ajustar o espaçamento dos contornos
            "color": "black",
        }
    }
)

# Criar o gráfico 3D para a fórmula fornecida com contornos
surface_formula = go.Surface(
    x=Re_grid, y=Stdp_grid, z=f_formula, 
    colorscale='Plasma',
    name='Superfície Fórmula',
    opacity=0.7,
    colorbar=dict(title='Fórmula f', len=0.5, x=1.05),
    contours={
        "z": {
            "show": True,
            "start": np.min(f_formula),
            "end": np.max(f_formula),
            "size": (np.max(f_formula) - np.min(f_formula)) / 20,
            "color": "black",
        }
    }
)

# Scatter plot dos valores originais
scatter_data = go.Scatter3d(
    x=ANNDF.iloc[:, 1].values,  # Valores de Reynolds
    y=ANNDF.iloc[:, 3].values,  # Valores de Stdp
    z=ANNDF.iloc[:, 2].values,  # Valores de 'f'
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Valores Originais'
)

# Definir o layout do gráfico
layout = go.Layout(
    title='Superfície de Resultados da ANN e da Fórmula com Valores Originais',
    scene=dict(
        xaxis=dict(title='Reynolds Number (Re)', backgroundcolor='white', gridcolor='lightgrey'),
        yaxis=dict(title='Stdp', backgroundcolor='white', gridcolor='lightgrey'),
        zaxis=dict(title='f', backgroundcolor='white', gridcolor='lightgrey'),
        bgcolor='white'
    ),
    paper_bgcolor='white',
    plot_bgcolor='white',
    showlegend=True
)

# Criar a figura com ambas as superfícies e o scatter plot
fig = go.Figure(data=[surface_ann], layout=layout)

# Mostrar o gráfico
pyo.plot(fig)


'temp-plot.html'

In [47]:

y = (4.3059 / np.log(1843 + 3.1918)) + ((0.30649 / 0.4331)-0.80937)
print(y)

0.4708217299026761


In [51]:
from sklearn.metrics import r2_score

# Achatar as superfícies em vetores 1D
f_ann_flat = f_pred_grid.ravel()
f_formula_flat = f_formula.ravel()

# Calcular o R^2
r2 = r2_score(f_ann_flat, f_formula_flat)

print('R² entre a superfície da ANN e a fórmula: {:.4f}'.format(r2))


R² entre a superfície da ANN e a fórmula: 0.9568


In [122]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(1000, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)

# Calcular a superfície usando a fórmula fornecida
f_formula = (4.3059 / np.log(Re_grid + 3.1918)) + ((0.30649 / Stdp_grid) - 0.80937)

# Criar o gráfico de contorno 2D para a fórmula fornecida
contour_formula = go.Contour(
    z=f_formula, 
    x=Re_vals, 
    y=Stdp_vals, 
    colorscale='Viridis', 
    name='Fórmula Contour',
    colorbar=dict(title='f')
)

# Definir o layout do gráfico
layout_formula = go.Layout(
    title='f Contour (Formula)',
    xaxis=dict(title='Reynolds Number (Re)'),
    yaxis=dict(title='Stdp'),
    width=800,  # Largura da imagem
    height=600  # Altura da imagem
)

# Criar a figura com o gráfico de contorno da fórmula
fig_formula = go.Figure(data=[contour_formula], layout=layout_formula)

# Mostrar o gráfico
pio.show(fig_formula)


In [57]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(1000, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# Prever os valores de 'f' usando a ANN nos pontos da grade
f_pred_scaled = gsearchcv.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

# Criar o gráfico de contorno 2D para os resultados da ANN
contour_ann = go.Contour(
    z=f_pred_grid, 
    x=Re_vals, 
    y=Stdp_vals, 
    colorscale='Viridis', 
    name='ANN Contour',
    colorbar=dict(title='ANN f')
)

# Scatter plot dos valores originais
scatter_data = go.Scatter(
    x=ANNDF.iloc[:, 1].values,  # Valores de Reynolds
    y=ANNDF.iloc[:, 3].values,  # Valores de Stdp
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Valores Originais'
)

# Definir o layout do gráfico
layout_ann = go.Layout(
    title='Contorno dos Resultados da ANN com Valores Originais',
    xaxis=dict(title='Reynolds Number (Re)'),
    yaxis=dict(title='Stdp'),
    width=800,  # Largura da imagem
    height=600  # Altura da imagem
)

# Criar a figura com o gráfico de contorno da ANN e o scatter plot dos valores originais
fig_ann = go.Figure(data=[contour_ann], layout=layout_ann)

# Mostrar o gráfico
pio.show(fig_ann)
